In [1]:
%%html
<style>.text_cell .rendered_html * {direction: rtl; text-align: right;}</style>

## هدف

در این پروژه می خواهیم با استفاده از الگوریتم های سرچ آگاهانه BFS و IDS و همچنین الگوریتم های سرچ نا آگاهانه A* و weighted A star راه حل مناسب برای مسئله پیده کنیم.
ویژگی
 های هر کدام و مزایا و معایب هر کدام بررسی خواهد شد.

In [2]:
import time
from heapq import heappop, heappush
import math

ابتدا ورودی ها را با استفاده از متد get_info از فایل می گیریم و اطلاعات مسئله را در کلاس Maze ذخیره می کنیم.

In [3]:
def get_info(_data):
    lines = _data.splitlines()
    n, m = lines[0].split()
    n = int(n)
    m = int(m)
    start = lines[1].split()
    start = (int(start[0]), int(start[1]))
    end = lines[2].split()
    end = (int(end[0]), int(end[1]))
    c = int(lines[3])
    k = int(lines[4])
    balls = {}
    map = []
    for i in range(k):
        temp = lines[i + 5].split()
        temp = [int(j) for j in temp]
        balls[(temp[0], temp[1])] = (temp[2], temp[3])
    balls_0 = list(balls.keys())
    balls_1 = list(balls.values())
    for i in range(n):
        temp = lines[i + k + 5].split()
        map.append(temp)
    for i in range(n):
        for j in range(m):
            if map[i][j] == '*':
                map[i][j] = 1
            if map[i][j] == '-':
                map[i][j] = 0
    return m, n, c, start, end, balls, map, balls_0, balls_1


## مسئله: Maze

کلاس Maze سه متد دارد:  
get_initial_state: برای الگوریتم های سرچ زمانی که می خواهیم استیت اولیه را ایجاد کنیم به دلیل این که کاملا به اطلاعات مسئله وابسته است بنابر این به عهده این کلاس است.  
goal_test: شرایط پایان مسئله را بررسی می کند که در صورت برقرار بودن آن شرایط خروجی True  است.  
actions: به دلیل این که اکشن های هر مسيله مختص به خود آن مسئله است بنابر این این متد را هم در کلاس Maze ذخیره می کنیم.

In [4]:
class Maze:
    def __init__(self, m, n, c, start, end, balls, map, balls_0, balls_1):
        self.m = m
        self.n = n
        self.c = c
        self.start = start
        self.end = end
        self.balls = balls
        self.map = map
        self.balls_0 = balls_0
        self.balls_1 = balls_1

    def get_initial_state(self):
        backpack = set()
        keys = self.balls.keys()
        ball_remain_loc = set(keys)
        return State(ball_remain_loc, backpack, self.start)

    def goal_test(self, state):
        return len(state.ball_remain_loc) == 0 and state.agent_loc == self.end and len(state.backpack) == 0

    def actions(self):
        return ['take_ball', 'left', 'up', 'right', 'down']


## State

متد های __hash__ و __eq__ را override کردیم که بتوان برای وجود داشتن یک instance از state در set از آن استفاده کند.

متد put_ball در هر مرحله اگر توپی در کوله پشتی داشته باشد که مکان قرار گیری آن با جایی که agent اکنون در آن قرار دارد برابر بود، توپ را روی زمین قرار می دهد.

In [5]:
class State:
    def __init__(self, ball_remain_loc, backpack, agent_loc):
        self.ball_remain_loc = ball_remain_loc
        self.backpack = backpack
        self.agent_loc = agent_loc

    def __hash__(self):
        return hash((frozenset(self.ball_remain_loc), frozenset(self.backpack), tuple(self.agent_loc)))

    def __eq__(self, other):
        return  self.ball_remain_loc == other.ball_remain_loc and self.backpack == other.backpack and self.agent_loc == other.agent_loc

    def __ne__(self, other):
        return not self.__eq__(other)

    def put_ball(self, balls_0, balls_1):
        if self.agent_loc in balls_1:
            pos = balls_1.index(self.agent_loc)
            ball = balls_0[pos]
            if ball in self.backpack:
                self.backpack.remove(ball)

    def print(self):
        print(self.ball_remain_loc, self.backpack, self.agent_loc)


## Node

این کلاس یک attribute برای State دارد و به طور کلی برای نگه داشتن اطلاعات مربوط به state است. برای print کردن خروجی از این  کلاس استفاده می شود.
  
  متد __lt__ را overrideکردم که در الگوریتم A* که از heap استفاده می کنم، بر اساس تابع heuristic در heap عملیات push و pop را انجام دهد. در ادامه بیشتر توضیح خواهم داد.

در متد child_node هنگامی که یک نود به آن داده می شود با توجه به actionآن nodeو stateجدیدی با تغییرات اعمال شده ایجاد می کند و آن را بر می گرداند.  
در این بخش به دلیل این که set ها با وجود این که در متغیر جدیدی ریخته می شودند ولی همچنان by refrence هستند و باید کپی آن ها در متغیر ریخته شود که مقادیر state والد را تغییر ندهد.

In [6]:
class Node:
    def __init__(self, state, parent, action, path_cost):
        self.state = state
        self.parent = parent
        self.action = action
        self.path_cost = path_cost
        self.depth = None
        self.heuristic = None

    def __lt__(self, other):
        return self.heuristic <= other.heuristic

    def print_path(self):
        if self is not None:
            print('loc:', self.state.agent_loc, ' f:', self.heuristic, ' cost:', self.path_cost, ' action:', self.action)
        if self.parent is not None:
            return self.parent.print_path()
        else:
            return

    def child_node(self, maze, action):
        child_ball_remain_loc = self.state.ball_remain_loc.copy()
        child_backpack = self.state.backpack.copy()
        child_agent_loc = self.state.agent_loc

        if action == 'take_ball':
            if child_agent_loc in child_ball_remain_loc and len(child_backpack) < maze.c:
                child_ball_remain_loc.remove(self.state.agent_loc)
                child_backpack.add(self.state.agent_loc)
        if action == 'left':
            child_agent_loc = (self.state.agent_loc[0], self.state.agent_loc[1]-1)
        if action == 'right':
            child_agent_loc = (self.state.agent_loc[0], self.state.agent_loc[1]+1)
        if action == 'up':
            child_agent_loc = (self.state.agent_loc[0]-1, self.state.agent_loc[1])
        if action == 'down':
            child_agent_loc = (self.state.agent_loc[0]+1, self.state.agent_loc[1])

        if 0 <= child_agent_loc[0] < maze.n and 0 <= child_agent_loc[1] < maze.m and maze.map[child_agent_loc[0]][child_agent_loc[1]] != 1:
            child_state = State(child_ball_remain_loc, child_backpack, child_agent_loc)
            child = Node(child_state, self, action, self.path_cost + 1)
            return child
        return self


## BFS

الگوریتم BFS به صورت سطحی جلو می رود.
در این الگوریتم frontier لیستی است که در هر iteration از ابتدای آن pop می کنیم. درواقع به صورت FIFO می باشد یعنی اولین نودی که به frontier اضافه شده اولین نودی است که از frontier خارج می شود.

در پیاده سازی این الگوریتم به دلیل این   که frontier ها را لیستی از  Node ها گرفته بودم، هنگامی که برای اضافه کردن child به frontier باید چک می کرد که این node در frontierوجود دارد یا نه، از نظر زمانی برای تست ۳ به مشکل می خوردم.  
بنابر این تصمیم گرفتم هنگام اضافه کردن child به frontierآن را به explored هم اضافه کنم. در این صورت هر بار فقط کافی است که state های explored را بررسی کند. و چون اوردر زمانی برای وجود داشتن یک نود در set به طور میانگین o(1) است در صورتی که اگر می خواستم  state های  node frontier را در setجدایی ذخیره کنم این کار با o(n)  نجام می شد.  
تعداد  state های مجزای دیده شده برابر است با تعداد دفعاتی که while اجرا  می شود، به دلیل این که هنگام اضافه کردن به frontier نود های تکراری را اضافه نمی کنیم.  
و تعداد کل state ها برابر است با مجموع تعداد همه استیت ها بدون توجه به این که در explored یا frontierوجود داشته اند.  
فاصله جواب  همان cost از نود مبدا تا نود مقصد می باشد.

In [7]:
def BFS(maze):
    initial_state = maze.get_initial_state()
    initial_node = Node(initial_state, None, None, 0)
    initial_node.total_state = 1
    if maze.goal_test(initial_node.state):
        return initial_node
    frontier = [initial_node]
    explored = set()
    explored.add(initial_state)
    unique_states = 0
    total_states = 0
    while True:
        unique_states += 1
        if not frontier:
            print("empty frontier")
            break
        node = frontier.pop(0)
        node.state.put_ball(maze.balls_0, maze.balls_1)
        for action in maze.actions():
            total_states += 1
            child = node.child_node(maze, action)
            if child.state not in explored:
                if maze.goal_test(child.state):
                    return child, unique_states, total_states
                frontier.append(child)
                explored.add(child.state)
    return None


# test BFS

In [8]:
data = open('1.txt').read()
m, n, c, start, end, balls, map, balls_0, balls_1 = get_info(data)
maze = Maze(m, n, c, start, end, balls, map, balls_0, balls_1)
tic = time.time()
solution, unique_states, total_states = BFS(maze)
toc = time.time()
if solution is not None:
    print("path is:")
    solution.print_path()
    print('unique states: ', unique_states)
    print('total states: ', total_states)
print("Run Time BFS: ", toc-tic)

path is:
loc: (2, 6)  f: None  cost: 20  action: right
loc: (2, 5)  f: None  cost: 19  action: down
loc: (1, 5)  f: None  cost: 18  action: right
loc: (1, 4)  f: None  cost: 17  action: right
loc: (1, 3)  f: None  cost: 16  action: right
loc: (1, 2)  f: None  cost: 15  action: take_ball
loc: (1, 2)  f: None  cost: 14  action: right
loc: (1, 1)  f: None  cost: 13  action: left
loc: (1, 2)  f: None  cost: 12  action: left
loc: (1, 3)  f: None  cost: 11  action: up
loc: (2, 3)  f: None  cost: 10  action: up
loc: (3, 3)  f: None  cost: 9  action: right
loc: (3, 2)  f: None  cost: 8  action: take_ball
loc: (3, 2)  f: None  cost: 7  action: left
loc: (3, 3)  f: None  cost: 6  action: down
loc: (2, 3)  f: None  cost: 5  action: take_ball
loc: (2, 3)  f: None  cost: 4  action: down
loc: (1, 3)  f: None  cost: 3  action: right
loc: (1, 2)  f: None  cost: 2  action: right
loc: (1, 1)  f: None  cost: 1  action: right
loc: (1, 0)  f: None  cost: 0  action: None
unique states:  238
total states:  1

## IDS

در این الگوریتم از limit صفر شروع می کنیم و در هر مرحله تا آن عمق خاص DFS را اجرا می کنیم.  
DFS همانند BFS می باشد با این تفاوت که در هر iteration نودی pop ی شود از انتهای frontier می باشد درواقع DFS به صورت LIFO می باشد. یعنی آخرین نودی که به فرانتیر وارد می شود اولین نودی است که خارج می شود.  

بهتر است که این الگوریتم را به صورت بازگشتی پیاده سازی کرد زیرا در غیر این صورت مشکل زیر پیش می آید:
مثلا اگر همچین شرایطی  را در نظر بگیریم: 

  
  اگر limit  را ۳ قرار دهیم ابتدا نود A سپس Bو بعد Cبررسی می شود و  پس از backtracking به S , سپس D به دلیل این که نود C را به explored اضافه کردیم، دیگر دیگر آن را وارد frontier نمی کند. بنابر این این الگوریتم بهینه ترین جواب راا زمانی به ما می دهد که state  های ویزیت شده را هنگام backtracking از اکسپلور خارج کنیم.
 که بهترین روش زدن تابع به صورت بازگشتی است.

پیاده سازی این الگوریتم به صورت غیر بازگشتی:

In [9]:
def IDS(maze):
    limit = 1
    initial_state = maze.get_initial_state()
    initial_node = Node(initial_state, None, None, 0)
    unique_states = 0
    total_states = 0
    while True:
        initial_node.depth = 1
        frontier = [initial_node]
        explored = set()
        explored.add(initial_node.state)
        result, unique_states, total_states = DFS(maze, limit, explored, frontier, unique_states, total_states)
        if result is not None:
            return result, unique_states, total_states
        limit = limit + 1
def DFS(maze, limit, explored, frontier, unique_states, total_states):
    while True:
        # print('here')
        if not frontier:
            return None, unique_states, total_states
        unique_states += 1
        node = frontier.pop()
        if node.state in explored and node.depth >= limit:
            continue
        if maze.goal_test(node.state):
            return node, unique_states, total_states
        for action in maze.actions():
            total_states += 1
            child = node.child_node(maze, action)
            child.state.put_ball(maze.balls_0, maze.balls_1)
            if child.state not in explored:
                child.depth = node.depth + 1
                explored.add(child.state)
                frontier.append(child)


# test IDS

In [10]:
data = open('2.txt').read()
m, n, c, start, end, balls, map, balls_0, balls_1 = get_info(data)
maze = Maze(m, n, c, start, end, balls, map, balls_0, balls_1)
tic = time.time()
solution, unique_states, total_states = IDS(maze)
toc = time.time()
if solution is not None:
    print("path is:")
    solution.print_path()
    print('unique states: ', unique_states)
    print('total states: ', total_states)
print("Run Time BFS: ", toc-tic)

path is:
loc: (48, 48)  f: None  cost: 48  action: right
loc: (48, 47)  f: None  cost: 47  action: right
loc: (48, 46)  f: None  cost: 46  action: right
loc: (48, 45)  f: None  cost: 45  action: right
loc: (48, 44)  f: None  cost: 44  action: right
loc: (48, 43)  f: None  cost: 43  action: right
loc: (48, 42)  f: None  cost: 42  action: right
loc: (48, 41)  f: None  cost: 41  action: right
loc: (48, 40)  f: None  cost: 40  action: right
loc: (48, 39)  f: None  cost: 39  action: right
loc: (48, 38)  f: None  cost: 38  action: right
loc: (48, 37)  f: None  cost: 37  action: right
loc: (48, 36)  f: None  cost: 36  action: right
loc: (48, 35)  f: None  cost: 35  action: right
loc: (48, 34)  f: None  cost: 34  action: right
loc: (48, 33)  f: None  cost: 33  action: right
loc: (48, 32)  f: None  cost: 32  action: right
loc: (48, 31)  f: None  cost: 31  action: right
loc: (48, 30)  f: None  cost: 30  action: right
loc: (48, 29)  f: None  cost: 29  action: right
loc: (48, 28)  f: None  cost: 2

## A*

این الگوریتم مسیر یابی با تخمین زدن مسیر تا رسیدن به هدف، جست و جو را بهبود می دهد. این الگوریتم تلاش می کند که به . 
سمت هدف حرکت کند

در ادامه توضیحات بیشتری راجع به الگوریتم و پیاده سازی آن آمده است.

برای این الگوریتم روش های متفاوتی مانند ذخبره frontier به صورت dict که مقدار heuristicرا هم در خود ذخیره کند و همچنین این که heuristic را در یک لیست جدا ذخیره کنم را امتحان کردم. ولی همه این روش ها با تایم زیادی به جواب می رسید.  
بنابر این بهترین روش استفاده از heap برای frontierاست. در این روش برای هر نود heuristic متناظر با آن را محاسبه کرده و در خود node به عنوان یک attribute از آن ذخیره می کند. و هنگام pop کردن نودی را که کمتیرین heuristic را دارد pop می کند. 

برای این سوال من چند heuristic در نظر گرفتم که به توضیح آن ها می پردازیم:


heuristic1:  
در این تابع برای همه توپ های کوله پشتی فاصله اقلیدسی آن ها از جایی که ایجنت قرار دارد تا مقصد توپ ها و  برای توپ هایی یکه روی زمین قرار دارند فاصله مبدا تا مقصد توپ و کل این مقدار به اضافه فاصله مکان ایجنت تا نقطه پایانی را در نظر گرفتم 
این تابع هیرویستیک admissible می باشد به دلیل این که فاصله تخمین زده شده حتما از فاصله واقعی کمتر است

اگر تابع consistent باشد به این صورت است که اگر فرض کنیم دو نود A , C داریم هزینه واقعی A تا پایان حتما از هزینه تخمین زده شده از A به C و از C به پایان کمتر می باشد.

همچنین در توابع herusistic ای که consistentهستند تابع f یعنی مجموع هزینه واقعی از state ابتدایی تا آن جا به اضافه تخمین هزینه از آن state تا state نهایی همواره غیر کاهشی است.


In [11]:
def heuristic1(maze, node):
    result = 1
    for ball_from in node.state.backpack:
        ball_to = maze.balls_1[maze.balls_0.index(ball_from)]
        result += (math.sqrt((node.state.agent_loc[0] - ball_to[0]) ** 2 + (node.state.agent_loc[1] - ball_to[1]) ** 2))
    for ball_from in node.state.ball_remain_loc:
        ball_to = maze.balls_1[maze.balls_0.index(ball_from)]
        # result += (math.sqrt((node.state.agent_loc[0] - ball_from[0])**2 + (node.state.agent_loc[1] - ball_from[1])**2))
        result += (math.sqrt((ball_from[0] - ball_to[0]) ** 2 + (ball_from[1] - ball_to[1]) ** 2))
    result += abs(node.state.agent_loc[0] - maze.end[0]) + abs(node.state.agent_loc[1] - maze.end[1])

    return result


heuristic2:  
این تابع مشابه توضیحات قبل با این تفاوت که فاصله منهتن را در نظر گرفتم.

In [12]:
def heuristic2(maze, node):
    result = 1
    for ball_from in node.state.backpack:
        ball_to = maze.balls_1[maze.balls_0.index(ball_from)]
        result += abs(node.state.agent_loc[0]-ball_to[0]) + abs(node.state.agent_loc[1] - ball_to[1])
    for ball_from in node.state.ball_remain_loc:
        ball_to = maze.balls_1[maze.balls_0.index(ball_from)]
#         result += abs(node.state.agent_loc[0]-ball_from[0]) + abs(node.state.agent_loc[1] - ball_from[1])
        result += abs(ball_from[0]-ball_to[0]) + abs(ball_from[1] - ball_to[1])
    result += abs(node.state.agent_loc[0] - maze.end[0]) + abs(node.state.agent_loc[1] - maze.end[1])
    return result


heuristic3:  
در این تابع فقط فاصله ایجنت تا خروج را در نظر می گیریم.

In [13]:
def heuristic3(maze, node):
    result = 1
    result += abs(node.state.agent_loc[0] - maze.end[0]) + abs(node.state.agent_loc[1] - maze.end[1])
    return result


کد الگوریتم مشابه BFS است با این تفاوت که در هر iteration نودی را از frontier pop می کنیم که کمترین مقدار fرا دااشته باشد.  
*: فیلدی که با عنوان  heurisistic  برای node ذخیره کرده ام درواقع همان f است یعنی h + g که مجموع هزینه واقعی تا آن state  و هزینه تخمین زده شده تا انتها می باشد.

In [14]:
def aStar(maze):
    initial_state = maze.get_initial_state()
    node = Node(initial_state, None, None, 0)
    if maze.goal_test(node.state):
        return node
    frontier = [node]
    explored = set()
    explored.add(node.state)
    unique_states = 0
    total_states = 0
    while True:
        unique_states += 1
        if not frontier:
            break
        node = heappop(frontier)
        node.state.put_ball(maze.balls_0, maze.balls_1)
        for action in maze.actions():
            total_states += 1
            child = node.child_node(maze, action)
            child.heuristic = heuristic3(maze, child) + child.path_cost
            if child.state not in explored:
                if maze.goal_test(child.state):
                    return child, unique_states, total_states
                heappush(frontier, child)
                explored.add(child.state)
    return None

#  test   aStar

In [15]:
data = open('1.txt').read()
m, n, c, start, end, balls, map, balls_0, balls_1 = get_info(data)
maze = Maze(m, n, c, start, end, balls, map, balls_0, balls_1)
tic = time.time()
solution, unique_states, total_states = aStar(maze)
toc = time.time()
if solution is not None:
    print("path is:")
    solution.print_path()
    print('unique states: ', unique_states)
    print('total states: ', total_states)
print("Time of calculating BFS: ", toc-tic)

path is:
loc: (2, 6)  f: 21  cost: 20  action: right
loc: (2, 5)  f: 21  cost: 19  action: right
loc: (2, 4)  f: 21  cost: 18  action: right
loc: (2, 3)  f: 21  cost: 17  action: down
loc: (1, 3)  f: 21  cost: 16  action: right
loc: (1, 2)  f: 21  cost: 15  action: take_ball
loc: (1, 2)  f: 20  cost: 14  action: right
loc: (1, 1)  f: 20  cost: 13  action: left
loc: (1, 2)  f: 18  cost: 12  action: left
loc: (1, 3)  f: 16  cost: 11  action: up
loc: (2, 3)  f: 14  cost: 10  action: up
loc: (3, 3)  f: 14  cost: 9  action: right
loc: (3, 2)  f: 14  cost: 8  action: take_ball
loc: (3, 2)  f: 13  cost: 7  action: left
loc: (3, 3)  f: 11  cost: 6  action: down
loc: (2, 3)  f: 9  cost: 5  action: take_ball
loc: (2, 3)  f: 8  cost: 4  action: down
loc: (1, 3)  f: 8  cost: 3  action: right
loc: (1, 2)  f: 8  cost: 2  action: right
loc: (1, 1)  f: 8  cost: 1  action: right
loc: (1, 0)  f: 8  cost: 0  action: None
unique states:  199
total states:  994
Time of calculating BFS:  0.01079821586608886

همان طور که مشاهده می شود مقدار f در طول مسیرغیر کاهشی می باشد.  
بنابر این heuristic3 consistentمی باشد   
فکر می کردم که heuristic1 و ۲ هم consistent  باشد ولی با مشاهده f آن متوجه می شویم که مقدار آن در مسیر گاهی کاهشی هم می شود.

In [16]:
data = open('2.txt').read()
m, n, c, start, end, balls, map, balls_0, balls_1 = get_info(data)
maze = Maze(m, n, c, start, end, balls, map, balls_0, balls_1)
tic = time.time()
solution, unique_states, total_states = aStar(maze)
toc = time.time()
if solution is not None:
    print("path is:")
    solution.print_path()
    print('unique states: ', unique_states)
    print('total states: ', total_states)
print("Time of calculating BFS: ", toc-tic)

path is:
loc: (48, 48)  f: 49  cost: 48  action: right
loc: (48, 47)  f: 49  cost: 47  action: right
loc: (48, 46)  f: 49  cost: 46  action: right
loc: (48, 45)  f: 49  cost: 45  action: right
loc: (48, 44)  f: 49  cost: 44  action: right
loc: (48, 43)  f: 49  cost: 43  action: right
loc: (48, 42)  f: 49  cost: 42  action: right
loc: (48, 41)  f: 49  cost: 41  action: right
loc: (48, 40)  f: 49  cost: 40  action: right
loc: (48, 39)  f: 49  cost: 39  action: right
loc: (48, 38)  f: 49  cost: 38  action: right
loc: (48, 37)  f: 49  cost: 37  action: right
loc: (48, 36)  f: 49  cost: 36  action: right
loc: (48, 35)  f: 49  cost: 35  action: right
loc: (48, 34)  f: 49  cost: 34  action: right
loc: (48, 33)  f: 49  cost: 33  action: right
loc: (48, 32)  f: 49  cost: 32  action: right
loc: (48, 31)  f: 49  cost: 31  action: right
loc: (48, 30)  f: 49  cost: 30  action: right
loc: (48, 29)  f: 49  cost: 29  action: right
loc: (48, 28)  f: 49  cost: 28  action: right
loc: (48, 27)  f: 49  cos

مشاهده می شود که برای  تست ۲ تعداد state های مجزای دیده شده برابر با همان طول مسیر اصلی می باشد. درواقع در این تست ایجنت مستقیما به سمت هدف حرکت کرده است.

## weighted aStar

در این الگوریتم برای تابع heuristic ضریبی بزرگ تر از یک در نظر می گیریم  و در واقع تاثیر تخمین هزینه تا state نهایی  را بیشتر از هزینه واقعی از state ابتدایی تا stateفعلی در نظر می گیریم.  و قدرت بیشتری در تاثیر گذاری در انتخاب state بعدی که می خواهد از frontier pop شود دارد.

In [17]:
def WaStar(maze, alpha):
    initial_state = maze.get_initial_state()
    node = Node(initial_state, None, None, 0)
    if maze.goal_test(node.state):
        return node
    frontier = [node]
    explored = set()
    explored.add(node.state)
    unique_states = 0
    total_states = 0
    while True:
        unique_states += 1
        if not frontier:
            break
        node = heappop(frontier)
        node.state.put_ball(maze.balls_0, maze.balls_1)
        for action in maze.actions():
            total_states += 1
            child = node.child_node(maze, action)
            child.heuristic = alpha*heuristic1(maze, child) + child.path_cost
            if child.state not in explored:
                if maze.goal_test(child.state):
                    return child, unique_states, total_states
                heappush(frontier, child)
                explored.add(child.state)
    return None

# test weighted aStar alpha = 2

In [18]:
data = open('1.txt').read()
m, n, c, start, end, balls, map, balls_0, balls_1 = get_info(data)
maze = Maze(m, n, c, start, end, balls, map, balls_0, balls_1)
tic = time.time()
solution, unique_states, total_states = WaStar(maze, 2)
toc = time.time()
if solution is not None:
    print("path is:")
    solution.print_path()
    print('unique states: ', unique_states)
    print('total states: ', total_states)
print("Time of calculating BFS: ", toc-tic)

path is:
loc: (2, 6)  f: 26  cost: 24  action: right
loc: (2, 5)  f: 27  cost: 23  action: right
loc: (2, 4)  f: 28  cost: 22  action: right
loc: (2, 3)  f: 29  cost: 21  action: down
loc: (1, 3)  f: 30  cost: 20  action: right
loc: (1, 2)  f: 31  cost: 19  action: right
loc: (1, 1)  f: 32  cost: 18  action: left
loc: (1, 2)  f: 31.0  cost: 17  action: left
loc: (1, 3)  f: 30.0  cost: 16  action: up
loc: (2, 3)  f: 27.47213595499958  cost: 15  action: up
loc: (3, 3)  f: 29.65685424949238  cost: 14  action: right
loc: (3, 2)  f: 29.47213595499958  cost: 13  action: take_ball
loc: (3, 2)  f: 28.47213595499958  cost: 12  action: left
loc: (3, 3)  f: 25.47213595499958  cost: 11  action: down
loc: (2, 3)  f: 24.47213595499958  cost: 10  action: take_ball
loc: (2, 3)  f: 23.47213595499958  cost: 9  action: left
loc: (2, 4)  f: 20.47213595499958  cost: 8  action: left
loc: (2, 5)  f: 17.47213595499958  cost: 7  action: right
loc: (2, 4)  f: 20.47213595499958  cost: 6  action: down
loc: (1, 4)

## test weighted aStar alpha = 1.3

In [19]:
data = open('1.txt').read()
m, n, c, start, end, balls, map, balls_0, balls_1 = get_info(data)
maze = Maze(m, n, c, start, end, balls, map, balls_0, balls_1)
tic = time.time()
solution, unique_states, total_states = WaStar(maze, 1.3)
toc = time.time()
if solution is not None:
    print("path is:")
    solution.print_path()
    print('unique states: ', unique_states)
    print('total states: ', total_states)
print("Time of calculating BFS: ", toc-tic)

path is:
loc: (2, 6)  f: 21.3  cost: 20  action: right
loc: (2, 5)  f: 21.6  cost: 19  action: right
loc: (2, 4)  f: 23.2  cost: 18  action: down
loc: (1, 4)  f: 24.038477631085023  cost: 17  action: right
loc: (1, 3)  f: 25.406888370749726  cost: 16  action: right
loc: (1, 2)  f: 26.910960958218894  cost: 15  action: take_ball
loc: (1, 2)  f: 25.910960958218894  cost: 14  action: right
loc: (1, 1)  f: 26.210960958218894  cost: 13  action: left
loc: (1, 2)  f: 25.210960958218894  cost: 12  action: left
loc: (1, 3)  f: 24.210960958218894  cost: 11  action: up
loc: (2, 3)  f: 22.21784932896862  cost: 10  action: up
loc: (3, 3)  f: 23.287916220388944  cost: 9  action: right
loc: (3, 2)  f: 22.81784932896862  cost: 8  action: take_ball
loc: (3, 2)  f: 21.81784932896862  cost: 7  action: left
loc: (3, 3)  f: 19.517849328968623  cost: 6  action: down
loc: (2, 3)  f: 18.517849328968623  cost: 5  action: take_ball
loc: (2, 3)  f: 17.517849328968623  cost: 4  action: down
loc: (1, 3)  f: 17.817

# نتیجه و مقایسه:

در الگوریتم A* تعداد استیت های دیده شده ودر نتیجه زمان اجرای الگوریتم نسبت به دو الگوریتم BFS و IDS کاهش میابد.  
در A* وزن دار با آلفا ۲ بهترین نتیجه را داریم  و آلفا ۱.۳ نتیجه بهتر از الگوریتم هایBFS و IDSولی بدتر از  الگوریتم A* با آلفا ۲ می دهد.  
درواقع در این جا با افزایش آلفا عملکرد بهتر شده است.

# Refrences:
https://stackoverflow.com/

In [20]:
import os
os.system('jupyter nbconvert --to html AI_CA1_810896059.ipynb')

0